In [1]:
from datasets import load_dataset
from sklearn.metrics import v_measure_score

dataset = load_dataset("tattabio/e_coli_rnas")  # returns a Hugging Face dataset
print(dataset["train"][0])

{'Sequence': 'GGCGGCCGTAGCGCGGTGGTCCCACCTGACCCCATGCCGAACTCAGAAGTGAAACGCCGTAGCGCCGATGGTAGTGTGGGGTCTCCCCATGCGAGAGTAG', 'Label': 'rRNA'}


In [4]:
import subprocess
from collections import defaultdict
from datasets import load_dataset
import pandas as pd
from sklearn.metrics import v_measure_score
import shutil
import sys

def save_fasta(dataset, output_path):
    with open(output_path, "w") as f:
        for i, sample in enumerate(dataset):
            seq_id = f"seq{i}"
            sequence = sample["Sequence"].replace("U", "T")  # Just in case
            f.write(f">{seq_id}\n{sequence}\n")


def run_cd_hit_est(input_fasta, output_fasta, identity=0.9, threads=4, memory=8000):
    # Check if cd-hit-est is installed
    if shutil.which("cd-hit-est") is None:
        print("❌ Error: 'cd-hit-est' is not installed or not in your PATH.")
        print("👉 Please install CD-HIT manually from https://github.com/weizhongli/cdhit/releases")
        print("   and add it to your system PATH.")
        print("The command was:")
        print(f"cd-hit-est -i {input_fasta} -o {output_fasta} -c {identity} -n 8 -T {threads} -M {memory}")
        sys.exit(1)

    # Run cd-hit-est
    try:
        subprocess.run([
            "cd-hit-est",
            "-i", input_fasta,
            "-o", output_fasta,
            "-c", str(identity),
            "-n", "8",
            "-T", str(threads),
            "-M", str(memory)
        ], check=True)
    except subprocess.CalledProcessError as e:
        print("❌ CD-HIT execution failed.")
        print(f"Error: {e}")
        sys.exit(1)

def parse_clstr_file(clstr_path):
    clusters = defaultdict(list)
    current_cluster = None
    with open(clstr_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith(">Cluster"):
                current_cluster = line.split()[-1]
            else:
                seq_id = line.split(">")[1].split("...")[0]
                clusters[current_cluster].append(seq_id)
    return clusters

def compute_v_score(clusters, labels):
    # Flatten clusters
    cluster_assignments = []
    for cluster_id, members in clusters.items():
        for seq_id in members:
            cluster_assignments.append((seq_id, cluster_id))
    cluster_df = pd.DataFrame(cluster_assignments, columns=["id", "cluster_id"])

    # Create label dataframe
    label_df = pd.DataFrame([
        (f"seq{i}", sample["Label"]) for i, sample in enumerate(labels)
    ], columns=["id", "true_label"])

    # Merge
    merged = pd.merge(cluster_df, label_df, on="id")
    v = v_measure_score(merged["true_label"], merged["cluster_id"])
    print(f"✅ V-score: {v:.4f}")
    return v

In [ ]:
dataset = load_dataset("tattabio/e_coli_rnas")["train"]

fasta_file = "ecoli_rna.fasta"
clustered_file = "ecoli_rna_clustered.fa"
clstr_file = clustered_file + ".clstr"

# Step 1: Save sequences to FASTA
save_fasta(dataset, fasta_file)

# Step 2: Run CD-HIT
# run_cd_hit_est(fasta_file, clustered_file, identity=0.9) # running cd-hit on wsl on windows, Shamy can try on his macbook

# Step 3: Parse clusters
clusters = parse_clstr_file(clstr_file)

# Step 4: Compute V-score
compute_v_score(clusters, dataset)

✅ V-score: 0.3045


0.30446556760185245